In [15]:
import torch
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
from scipy.linalg import sqrtm

In [16]:
# Choose GPU if available, otherwise fallback to CPU
if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
else:
    DEVICE = torch.device("cpu")
print(DEVICE)

cuda


In [45]:
def compute_V_aux(V1, V2):
    n = int(len(V1) / 2)  # Number of modes (assumes square matrices of size 2n x 2n)
    
    # Step 1: Define the correct 12x12 symplectic matrix Omega for n=6
    I_n = np.eye(n)  # Create the identity matrix I_n (6x6 for 6 modes)
    
    # Omega = [0 I_n; -I_n 0], block construction
    Omega = np.zeros((2*n, 2*n))
    Omega[:n, n:] = I_n  # Upper right block is I_n
    Omega[n:, :n] = -I_n  # Lower left block is -I_n


    # Step 4: Compute V1 + V2 and its inverse
    V_sum = V1 + V2
    V_inv = np.linalg.inv(V_sum)
    
    # Step 5: Compute the term (Ω / 4 + V2 * Ω * V1)
    symplectic_term = 0.25 * Omega + V2 @ Omega @ V1
    
    # Step 6: Compute V_aux
    V_aux = Omega.T @ V_inv @ symplectic_term
    
    return V_aux


In [50]:
def compute_F0(V1, V2, V_aux):
    # Step 1: Compute V1 + V2 and its determinant (denominator)
    V_sum = V1 + V2
    det_V_sum = np.linalg.det(V_sum)
    denominator = np.sqrt(np.sqrt(det_V_sum))
    
    # Step 2: Define Omega using Kronecker product for n modes
    n = int(len(V1) / 2)  # Number of modes
    # Step 1: Define the correct 12x12 symplectic matrix Omega for n=6
    I_n = np.eye(n)  # Create the identity matrix I_n (6x6 for 6 modes)
    
    # Omega = [0 I_n; -I_n 0], block construction
    Omega = np.zeros((2*n, 2*n))
    Omega[:n, n:] = I_n  # Upper right block is I_n
    Omega[n:, :n] = -I_n  # Lower left block is -I_n
    
    
    # Step 4: Compute V_aux * Omega
    V_aux_Omega = V_aux @ Omega
    
    # Step 5: Compute the inverse of V_aux * Omega
    V_aux_Omega_inv = np.linalg.inv(V_aux_Omega)
    
    # Step 6: Compute the matrix term (I + (V_aux * Omega)^(-2)/4)
    identity_matrix = np.eye(V_aux.shape[0])
    V_aux_Omega_inv_square = V_aux_Omega_inv @ V_aux_Omega_inv
    matrix_addition = identity_matrix + 0.25 * V_aux_Omega_inv_square
    
    # Step 7: Compute the matrix square root of the term
    matrix_sqrt = sqrtm(matrix_addition)
    
    # Step 8: Add the identity matrix I to the matrix square root
    matrix_sqrt_plus_identity = matrix_sqrt + identity_matrix
    
    # Step 9: Multiply by V_aux
    multiplication_result = matrix_sqrt_plus_identity @ V_aux
    
    # Step 10: Compute the determinant of the result and raise it to the 1/4 power (nominator)
    nominator = np.linalg.det(2 * multiplication_result) ** 0.25

    # Step 11: Compute F0
    F0 = nominator / denominator

    return F0

In [51]:
def fidelity(V1, mu1, V2, mu2):
    # Step 1: Compute V_aux first (V_aux = Omega.T @ (V_1 + V_2)^(-1) @ term)
    V_aux = compute_V_aux(V1, V2)
    
    # Step 2: Compute F0
    F0 = compute_F0(V1, V2, V_aux)
    
    # Step 3: Compute the displacement-dependent factor (exponential part)
    V_sum = V1 + V2
    V_inv = np.linalg.inv(V_sum)  # Compute the inverse of (V_1 + V_2)
    displacement_factor = np.exp(-0.25 * (mu1 - mu2).T @ V_inv @ (mu1 - mu2))  # Compute the exponential term
    
    # Step 4: Compute the final fidelity
    F = F0 * displacement_factor  # Multiply F0 by the displacement-dependent factor
    
    return F


In [100]:
# Example covariance matrix for state 1 (2x2)
V1 = np.array([[2, 1], [0, 1/2]])  # Example covariance matrix for state 1
V1 = V1 + V1.T  # Make the matrix symmetric

mu1 = np.array([5, 13])  # Example mean vector for state 1

# Example covariance matrix for state 2 (2x2)
V2 = np.array([[2, 1], [0, 1/2]])  # Example covariance matrix for state 2
V2 = V2 + V2.T  # Make the matrix symmetric

mu2 = np.array([5, 13])  # Example mean vector for state 2

# Compute fidelity for the system
fidelity_value = compute_fidelity(V1, mu1, V2, mu2)
print("Fidelity between the two Gaussian states:", fidelity_value)

Fidelity between the two Gaussian states: 0.9999999999999999


In [99]:
# Covariance matrix for state 1 (4x4)
V1 = np.array([[2, 1, 0, 0], [1, 2, 0, 0], [0, 0, 2, 1], [0, 0, 1, 2]])  # Example covariance matrix for state 1
V1 = V1 + V1.T  # Make the matrix symmetric

mu1 = np.array([5, 13, 5, 13])  # Example mean vector for state 1

# Covariance matrix for state 2 (4x4)
V2 = np.array([[2, 1, 0, 0], [1, 2, 0, 0], [0, 0, 2, 1], [0, 0, 1, 2]])  # Example covariance matrix for state 2
V2 = V2 + V2.T  # Make the matrix symmetric

mu2 = np.array([5, 13, 5, 13])  # Example mean vector for state 2

# Compute fidelity for the system
fidelity_value = compute_fidelity(V1, mu1, V2, mu2)
print("Fidelity between the two Gaussian states:", fidelity_value)

Fidelity between the two Gaussian states: 1.0
